<a href="https://colab.research.google.com/github/mickeykim70/udemy_tensorflow/blob/main/artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Colab path work



In [19]:
import os
os.chdir('/content/drive/MyDrive/Deep Learning A-Z/Part 1 - Artificial Neural Networks (ANN)')

# **Artificial Neural Network**

Importing the Libraries

In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [10]:
tf.__version__

'2.17.1'

## Part 1 - Data Preprocessing

### Importing dataset

In [24]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values
print(x[:5], y[:5])

[[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
 [608 'Spain' 'Female' 41 1 83807.86 1 0 1 112542.58]
 [502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]
 [699 'France' 'Female' 39 1 0.0 2 0 0 93826.63]
 [850 'Spain' 'Female' 43 2 125510.82 1 1 1 79084.1]] [1 0 1 0 0]


### Encoding Categorical data

#### Encoding the **Independent** variable

In [28]:
# Label Encoding for Gender column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2]) # Gender

In [29]:
x[:10]

array([[619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [645, 'Spain', 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [822, 'France', 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [376, 'Germany', 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [501, 'France', 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [684, 'France', 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

In [30]:
# Label Encoding for Geography column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

NameError: name 'X' is not defined